In [2]:
import pandas as pd
import csv
import itertools
import numpy as np

In [3]:
df = pd.read_csv('main_co_occur.csv', sep = ',')
df

,Query,Genus/Species,Nucleotide_acc,Protein_acc,start,end,dir,PfamID1,Name1,Description1,...,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50
0,WP_145853129.1,Trebonia kvetii,NZ_RPFW01000002.1,WP_187365872.1,891146,890102,-,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WP_145853129.1,Trebonia kvetii,NZ_RPFW01000002.1,WP_187365873.1,891477,891165,-,TIGR00377,TIGR00377,ant_ant_sig: anti-anti-sigma factor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WP_145853129.1,Trebonia kvetii,NZ_RPFW01000002.1,WP_145853125.1,891619,892345,+,PF02909,TetR_C_1,"Tetracyclin repressor-like, C-terminal domain",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WP_145853129.1,Trebonia kvetii,NZ_RPFW01000002.1,WP_145853126.1,892572,892368,-,PF00313,CSD,'Cold-shock' DNA-binding domain,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WP_145853129.1,Trebonia kvetii,NZ_RPFW01000002.1,WP_145853127.1,892834,892603,-,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11152,WP_015759610.1,Desulfofarcimen acetoxidans DSM 771,NC_013216.1,WP_015759614.1,4443259,4442230,-,PF00766,ETF_alpha,Electron transfer flavoprotein FAD-binding domain,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11153,WP_015759610.1,Desulfofarcimen acetoxidans DSM 771,NC_013216.1,WP_015759615.1,4445244,4443258,-,TIGR03997,TIGR03997,mycofact_OYE_2: mycofactocin system FadH/OYE f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11154,WP_015759610.1,Desulfofarcimen acetoxidans DSM 771,NC_013216.1,WP_015759616.1,4447200,4445256,-,TIGR03997,TIGR03997,mycofact_OYE_2: mycofactocin system FadH/OYE f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11155,WP_015759610.1,Desulfofarcimen acetoxidans DSM 771,NC_013216.1,WP_015759617.1,4447352,4447226,-,TIGR03969,TIGR03969,mycofactocin: mycofactocin precursor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
all_genes = df.loc[:,['Query', 'Name1', 'Description1']].dropna(how='all',subset = ['Name1']).drop_duplicates()
#выкидываю гены без названия

name = []
for i,row in all_genes.iterrows():
    if row['Name1'][:4]  == 'TIGR':
        name.append(row['Description1'].split(":")[0])
    else: 
        name.append(row['Name1'])
all_genes['Name'] = name
del all_genes['Name1']
del all_genes['Description1']
all_genes

,Query,Name
1,WP_145853129.1,ant_ant_sig
2,WP_145853129.1,TetR_C_1
3,WP_145853129.1,CSD
5,WP_145853129.1,OsmC
6,WP_145853129.1,mycofact_glyco
...,...,...
11151,WP_015759610.1,GG-red-SF
11152,WP_015759610.1,ETF_alpha
11153,WP_015759610.1,mycofact_OYE_2
11155,WP_015759610.1,mycofactocin


In [42]:
mftb_counts = dict()
for que in all_genes['Query'].drop_duplicates():
    buffer = all_genes[all_genes['Query'] == que]
    pair_order_list = itertools.combinations(buffer['Name'], 2)#все пары генов для данного query
    for pair in pair_order_list:
        if pair not in mftb_counts and pair[::-1] not in mftb_counts:
            mftb_counts[pair] = 1
        elif pair in mftb_counts:
            mftb_counts[pair] += 1
        elif pair[::-1] in mftb_counts:
            mftb_counts[pair[::-1]] += 1  #чтобы избавиться от ориентированных пар

mftb_counts = pd.DataFrame({'prot_1':[key[0] for key in mftb_counts.keys()],'prot_2':[key[1] for key in mftb_counts.keys()], 'counts':mftb_counts.values()})
mftb_counts = mftb_counts.sort_values(by = ['counts'], ascending=False)
mftb_counts
#mftb_counts['pairs'] = str[mftb_counts['pairs']]
#пары генов которые не встретились вместе(counts == 0) автоматически не учитываются

,prot_1,prot_2,counts
51,Mycofactocin_RRE,mycofactocin,646
46,mycofact_rSAM,mycofactocin,640
45,mycofact_rSAM,Mycofactocin_RRE,630
117,actino_HemFlav,mycofact_rSAM,475
118,actino_HemFlav,Mycofactocin_RRE,472
...,...,...,...
9787,AAA_5,ac_ac_CoA_syn,1
9788,VWA_CoxE,HTH_18,1
9789,VWA_CoxE,TIGR03086,1
9790,VWA_CoxE,ac_ac_CoA_syn,1


Ориентированные пары все еще могли остаться так как перебираются неориентированные пары для разных query, в которых перечисление генов мб разным. Если в одном query перечисление идет mycofactocin, Mycofactocin_RRE, ...
то в первую очередь будут генерироваться пары где на первом месте mycofactocin ==> будет пара (mycofactocin, Mycofactocin_RRE). 
Пусть в другом query перечисление идет так: Mycofactocin_RRE, mycofactocin...
тогда в первую очередь будут генерироваться пары где на первом месте Mycofactocin_RRE ==> будет пара (Mycofactocin_RRE, mycofactocin). 
в обоих кластерах они неориентированы, но при рассмотрении всех кластеров получаем ориентированные пары

In [46]:
#mftb_counts.to_csv('/home/marfa/Desktop/lab/Mycofactocin/mftb/cluster_res/mftb_pair_counts.csv',index=False,header=False,quotechar = "'")